In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
processed = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/rf_regressor.csv')

In [3]:
### Season to test results

N = 2022

In [ ]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [4]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()

X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [5]:
def get_predictions(X_train, y_train, model):
    points = []
    winners = []
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        points += list(prediction_df.predicted_points.values)
        winners += list(prediction_df.predicted_winner.values)

        # Retrain model
        X_train = pd.concat([X_train, X_test])
        y_train = np.append(y_train, y_test)

        model.fit(X_train, y_train)
        

    return points, winners

In [6]:
parameters.head()

,model,criterion,max_features,max_depth,score
0,random_forest_regressor,squared_error,0.8,51,0.631579
1,random_forest_regressor,squared_error,0.8,5,0.578947
2,random_forest_regressor,squared_error,1.0,21,0.578947
3,random_forest_regressor,squared_error,1.0,17,0.578947
4,random_forest_regressor,squared_error,1.0,27,0.578947


In [7]:
params = parameters.iloc[0]
criterion = params.criterion
max_features = params.max_features	
max_depth = params.max_depth

params

model           random_forest_regressor
criterion                 squared_error
max_features                        0.8
max_depth                            51
score                          0.631579
Name: 0, dtype: object

In [8]:
# Random Forest Regressor

test = processed.copy()
test = test[['season', 'round', 'driver', 'starting_grid', 'podium', 'driver_points_from', 'q_delta']].query('season ==@N')

model_params = (criterion, max_features, max_depth)
model = RandomForestRegressor(criterion=criterion, max_features=max_features, max_depth=max_depth)
model.fit(X_train, y_train)

points, winner = get_predictions(X_train, y_train, model)

test['predicted_points'] = points
test['predicted_winner'] = winner

In [9]:
test.query('predicted_winner == 1')

,season,round,driver,starting_grid,podium,driver_points_from,q_delta,predicted_points,predicted_winner
3111,2022,1,charles_leclerc,1,1,26.0,0.000,16.447452,1
3132,2022,2,charles_leclerc,2,2,19.0,0.025,18.870000,1
3149,2022,3,charles_leclerc,1,1,26.0,0.000,18.870000,1
3169,2022,5,charles_leclerc,1,2,18.0,0.000,17.600000,1
3187,2022,6,max_verstappen,2,1,25.0,0.323,19.680000,1
3210,2022,7,charles_leclerc,1,4,12.0,0.000,20.140000,1
3245,2022,8,charles_leclerc,1,19,0.0,0.000,17.920000,1
3247,2022,9,max_verstappen,1,1,25.0,0.000,21.810000,1
3273,2022,10,max_verstappen,2,7,6.0,0.072,19.370000,1
3287,2022,12,max_verstappen,2,1,25.0,0.304,17.170000,1


In [10]:
rounds = len(test['round'].unique())
correct = len(test.query('predicted_winner == 1 & podium <= 3'))

(correct / rounds) * 100

73.68421052631578